In [1]:
import os
from dotenv import load_dotenv
import googlemaps
import csv
import time 

load_dotenv('.env')

API_KEY = os.environ.get("API_KEY")
gmaps = googlemaps.Client(key=API_KEY)

In [2]:
area_name = 'Pimpri'
place_keyword = 'diagnostic center'
radius_in_km = 10

In [4]:
geocode_result = gmaps.geocode(area_name)
location = geocode_result[0]['geometry']['location']

places_result = gmaps.places_nearby(
    location=location,
    radius=radius_in_km*1000,
    keyword=place_keyword,
)

In [5]:
all_place_ids = []

all_place_ids.extend([place['place_id'] for place in places_result.get('results', [])])

# Check if there are more pages and retrieve them
while 'next_page_token' in places_result:
    next_page_token = places_result['next_page_token']
    
    # Pause for a short time to allow the next page to be generated
    time.sleep(2)

    # Make a request with the page token to get the next page of results
    places_result = gmaps.places_nearby(
        location=location,
        radius=radius_in_km * 1000,
        keyword=place_keyword,
        page_token=next_page_token
    )

    # Extract place IDs from the result and add them to the all_place_ids list
    all_place_ids.extend([place['place_id'] for place in places_result.get('results', [])])

In [7]:
csv_file_path = f"data/{len(all_place_ids)}_{place_keyword}_{area_name}_{radius_in_km}km.csv"
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Name', 'Address', 'Phone Number', 'URL'])

    # Write details to the CSV file
    for place_id in all_place_ids:
        # Fetch details for the current place ID
        place_details = gmaps.place(place_id=place_id, fields=['name', 'formatted_address', 'formatted_phone_number', 'url'])
        
        # Extract details
        name = place_details['result']['name']
        phone_number = place_details['result'].get('formatted_phone_number', '')
        address = place_details['result']['formatted_address']
        url = place_details['result'].get('url', '')

        # Write details to the CSV file
        csv_writer.writerow([name, address, phone_number, url])

print(f"Details saved to {csv_file_path}")

Details saved to data/60_diagnostic center_Pimpri_10km.csv
